In [ ]:
import os
import os.path as op
import sys

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

#from bluemath_tk.wrappers.swash.swash_wrapper import SwashModelWrapper
from IPython.display import HTML

from scripts.wrappers import SwashModelWrapper_ondas

from scripts.bathymetry import linear_profile
from scripts.plots import plot_case_config

## 01.Define Case Variables

#### Necessary Paths

In [ ]:
templates_dir = '/workspaces/ONDAS_Swash/templates/templates_test'
output_dir = '/workspaces/ONDAS_Swash/cases'

#### Hydrodynamic Forcing

In [ ]:
Hs = 2.5
Tp = 10
Hs_L0 =  round((2 * np.pi * (Hs)) / (9.81 * (Tp ** 2)),4)  # Should be between 0.005 and 0.05
WL = 0.0

In [ ]:
Hs_L0

#### Create Bathymetry

In [ ]:
h0 = -12  # offshore depth (m)
Ltotal = 1000  # beach heigh (m)
m = 10 / 300  # profile slope
Wfore = 400  # flume length before fore toe (m)

x_profile, depth_array = linear_profile(h0=h0, Ltotal=Ltotal, Wconst=Wfore, slope=m)

In [ ]:
plot_case_config(x=x_profile, z=depth_array, wave_height=Hs, wave_period = Tp, WL=WL)

In [ ]:
metamodel_parameters = pd.DataFrame({'Hs': Hs, 'Hs_L0': Hs_L0, 'WL': WL}, index=[0]).to_dict(orient='list')
fixed_parameters = {
    "dxinp": 1,  # bathymetry grid spacing  # Friction manning coefficient (m^-1/3 s)
    "comptime": 300,  # Simulation duration (s)
    "warmup": 300,  # Warmup duration (s)
    "n_nodes_per_wavelength": 60,  # number of nodes per wavelength
}

swash_model = SwashModelWrapper_ondas(
    templates_dir=templates_dir,
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=output_dir,
    depth_array=-depth_array,
)

In [ ]:
swash_model.build_cases(cases_name_format=lambda ctx: 'test')

In [ ]:
swash_model.run_cases(launcher='serial')

In [ ]:
vars_to_postprocess = ["Msetup", "Hrms", "Hfreqs"]
postprocessed = swash_model.postprocess_cases(output_vars=vars_to_postprocess).isel(case_num=0)

In [ ]:
postprocessed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axs = plt.subplots(1, 2, figsize=(12,4), sharey=True)

ax1 = axs[0]
ax1.plot(postprocessed.Xp, postprocessed.Hs, label='Hs', color='tab:blue', lw=2)
ax1.plot(postprocessed.Xp, postprocessed.Hss, label='Hss', color='tab:green', lw=2)
ax1.plot(postprocessed.Xp, postprocessed.Hrms, label='Hrms', color='tab:orange', lw=2)

ax1.fill_between(np.arange(0, len(depth_array)), depth_array, np.min(depth_array)-1,
                 color='navajowhite', alpha=0.4, zorder=0)
ax1.plot(np.arange(0, len(depth_array)), depth_array, color='k', lw=1.5, zorder=1)

ax1.set_ylabel('Wave Height (m)', fontsize=12)
ax1.grid(True, ls='--', alpha=0.4)
ax1.legend(loc='upper right', fontsize=10, ncol=2, frameon=True)

ax2 = axs[1]
ax2.plot(postprocessed.Xp, postprocessed.ig, label='IG', color='tab:red', lw=2)
ax2.plot(postprocessed.Xp, postprocessed.Msetup, label='Mean Setup', color='tab:brown', lw=2)

ax2.fill_between(np.arange(0, len(depth_array)), depth_array, np.min(depth_array)-1,
                 color='navajowhite', alpha=0.4, zorder=0)
ax2.plot(np.arange(0, len(depth_array)), depth_array, color='k', lw=1.5, zorder=1)

ax2.set_ylabel('Low-frequency Energy / Setup (m)', fontsize=12)
ax2.grid(True, ls='--', alpha=0.4)
ax2.legend(loc='upper right', fontsize=10, ncol=2, frameon=True)

axs[0].set_xlim([np.min(postprocessed.Xp), np.max(postprocessed.Xp)])

ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
           ncol=3, fontsize=10, frameon=False)
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
           ncol=2, fontsize=10, frameon=False)

fig.suptitle('Wave Transformation Across the Flume', fontsize=15)

plt.tight_layout()
plt.show()


In [ ]:
output_data = xr.open_dataset('/workspaces/ONDAS_Swash/cases/test/output.nc')

In [ ]:
import matplotlib.animation as animation


watlev = output_data["Watlev"]  # shape: (Xp, Yp, Tsec)
x = output_data["Xp"]
time = output_data["Tsec"]

watlev_1d = watlev.isel(Yp=0)

fig, ax = plt.subplots()
line, = ax.plot(x, watlev_1d.isel(Tsec=0), lw=2)

ax.fill_between(np.arange(0, len(depth_array)), depth_array, np.min(depth_array)-1,
                 color='navajowhite', alpha=0.4, zorder=0)
ax.plot(np.arange(0, len(depth_array)), depth_array, color='k', lw=1.5, zorder=1)


ax.set_xlim(x.min(), x.max())
ax.set_xlabel("X position")
ax.set_ylabel("Water Level (Watlev)")
title = ax.set_title("Time = 0 s")

# Animation function
def update(frame):
    line.set_ydata(watlev_1d.isel(Tsec=frame))
    title.set_text(f"Time = {time[frame].item()} s")
    return line, title

# Create the animation
ani = animation.FuncAnimation(
    fig, update, frames=len(time), interval=100, blit=True
)

plt.close()

HTML(ani.to_jshtml())